# Spark SQL

- Support ANSI SQL:2003 and HiveQL
- Run Queries
    - Spark SQL CLI : `./bin/spark-sql`
    - Spark SQL API : `spark.sql()`
    - Spark Thrift JDBC/ODBC Server and Client : `./sbin/start-thriftserver.sh` `./bin/beeline`

## Import modules

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import apache_access_log

## Session

In [2]:
spark = SparkSession.builder \
                    .master("local[4]") \
                    .appName("spark sql tutorial") \
                    .config("spark.executor.memory", "1g") \
                    .enableHiveSupport() \
                    .getOrCreate()

enableHiveSupport()
- connectivity to a persistent Hive metastore
- support for Hive serdes
- Hive user-defined functions.

## Context

In [3]:
sc = spark.sparkContext

## RDD

In [4]:
LOG_FILE_PATH = 'access.log'
rdd = sc.textFile(LOG_FILE_PATH) \
        .map(apache_access_log.parse) \
        .filter(lambda x: x is not None) 

## DataFrame

In [5]:
df = spark.createDataFrame(rdd)

## GlobalTempView (cross-session scope)

In [6]:
df.createGlobalTempView('global_temp_view')
#df.createOrReplaceGlobalTempView('global_temp_view')

In [7]:
!ls -l ./metastore_db ./derby.log
# for Hive metastore

-rw-r--r-- 1 root root 701 Aug 27 12:38 ./derby.log

./metastore_db:
total 16
-rw-r--r--   1 root root  608 Aug 19 16:45 README_DO_NOT_TOUCH_FILES.txt
-rw-r--r--   1 root root   38 Aug 27 12:38 db.lck
-rw-r--r--   1 root root    4 Aug 27 12:38 dbex.lck
drwxr-xr-x   6 root root  192 Aug 19 16:45 log
drwxr-xr-x 190 root root 6080 Aug 19 16:59 seg0
-rw-r--r--   1 root root  898 Aug 19 16:45 service.properties
drwxr-xr-x   2 root root   64 Aug 27 12:38 tmp


In [8]:
spark.sql('select * from global_temp.global_temp_view limit 1').show()

+--------------------+--------+---------------+--------+------+--------+-------+----+------+-------------------+---------------+----+
|               agent|    date|           host|identity|method|protocol|referer|size|status|               time|            url|user|
+--------------------+--------+---------------+--------+------+--------+-------+----+------+-------------------+---------------+----+
|Mozilla/5.0 (Wind...|20151212|109.169.248.247|       -|   GET|HTTP/1.1|      -|4263|   200|2015-12-12 17:25:11|/administrator/|   -|
+--------------------+--------+---------------+--------+------+--------+-------+----+------+-------------------+---------------+----+



In [9]:
spark2 = spark.newSession()
assert(spark!=spark2)
assert(spark.sparkContext==spark2.sparkContext)

In [10]:
spark2.sql("SELECT * FROM global_temp.global_temp_view limit 1").show()

+--------------------+--------+---------------+--------+------+--------+-------+----+------+-------------------+---------------+----+
|               agent|    date|           host|identity|method|protocol|referer|size|status|               time|            url|user|
+--------------------+--------+---------------+--------+------+--------+-------+----+------+-------------------+---------------+----+
|Mozilla/5.0 (Wind...|20151212|109.169.248.247|       -|   GET|HTTP/1.1|      -|4263|   200|2015-12-12 17:25:11|/administrator/|   -|
+--------------------+--------+---------------+--------+------+--------+-------+----+------+-------------------+---------------+----+



In [11]:
#spark.catalog.dropGlobalTempView('global_temp_view')

In [12]:
spark.sql('CREATE GLOBAL TEMPORARY VIEW global_temp_view2 AS SELECT 1 as a, 2 as b')

DataFrame[]

In [13]:
spark.sql('SELECT * FROM global_temp.global_temp_view2').show()

+---+---+
|  a|  b|
+---+---+
|  1|  2|
+---+---+



## TempView (current session scope)

In [14]:
df.createTempView('temp_view')
#df.createOrReplaceTempView("temp_view")

In [15]:
spark.sql('select * from temp_view limit 1').show()

+--------------------+--------+---------------+--------+------+--------+-------+----+------+-------------------+---------------+----+
|               agent|    date|           host|identity|method|protocol|referer|size|status|               time|            url|user|
+--------------------+--------+---------------+--------+------+--------+-------+----+------+-------------------+---------------+----+
|Mozilla/5.0 (Wind...|20151212|109.169.248.247|       -|   GET|HTTP/1.1|      -|4263|   200|2015-12-12 17:25:11|/administrator/|   -|
+--------------------+--------+---------------+--------+------+--------+-------+----+------+-------------------+---------------+----+



In [16]:
#spark2.sql('select * from temp_view limit 1').show() # throw exception

In [17]:
#spark.catalog.dropTempView('temp_view')

## Cache

In [18]:
assert(spark.catalog.isCached('temp_view')==False)

spark.catalog.cacheTable('temp_view')
assert(spark.catalog.isCached('temp_view')==True)

spark.catalog.uncacheTable('temp_view')
#spark.catalog.clearCache()
assert(spark.catalog.isCached('temp_view')==False)

spark.catalog.cacheTable('temp_view')

## Show Databases, Tables and Views

In [19]:
#print spark.catalog.listDatabases()
#print spark.catalog.listTables()
#print spark.catalog.listFunctions()
spark.sql('show databases').show()
spark.sql('show tables').show()
spark.sql('show functions').show()

+------------+
|databaseName|
+------------+
|     default|
+------------+

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|     test|      false|
|        |temp_view|       true|
+--------+---------+-----------+

+----------+
|  function|
+----------+
|         !|
|         %|
|         &|
|         *|
|         +|
|         -|
|         /|
|         <|
|        <=|
|       <=>|
|         =|
|        ==|
|         >|
|        >=|
|         ^|
|       abs|
|      acos|
|add_months|
| aggregate|
|       and|
+----------+
only showing top 20 rows



## SQL Query

### total pv

In [20]:
print df.count()
spark.sql("select count(1) from temp_view").show()

20000
+--------+
|count(1)|
+--------+
|   20000|
+--------+



### daily pv

In [21]:
df2 = df.groupBy('date').count().sort('date')
df2.cache()
spark.sql("select date, count(1) as count from temp_view group by date order by date").show()

+--------+-----+
|    date|count|
+--------+-----+
|20151212|  358|
|20151213| 1361|
|20151214|  996|
|20151215|  975|
|20151216| 1171|
|20151217|  929|
|20151218| 1307|
|20151219| 2189|
|20151220|  337|
|20151221|  521|
|20151222|  323|
|20151223|  271|
|20151224|  322|
|20151225|  216|
|20151226|  338|
|20151227|  504|
|20151228|  398|
|20151229|  729|
|20151230|  509|
|20151231|  394|
+--------+-----+
only showing top 20 rows



In [22]:
df2.show()

+--------+-----+
|    date|count|
+--------+-----+
|20151212|  358|
|20151213| 1361|
|20151214|  996|
|20151215|  975|
|20151216| 1171|
|20151217|  929|
|20151218| 1307|
|20151219| 2189|
|20151220|  337|
|20151221|  521|
|20151222|  323|
|20151223|  271|
|20151224|  322|
|20151225|  216|
|20151226|  338|
|20151227|  504|
|20151228|  398|
|20151229|  729|
|20151230|  509|
|20151231|  394|
+--------+-----+
only showing top 20 rows



### daily count of status

In [23]:
df.groupBy('date', 'status').count().sort('date', 'status').show()
spark.sql("select date, status, count(1) as count from temp_view group by date, status order by date, status").show()

+--------+------+-----+
|    date|status|count|
+--------+------+-----+
|20151212|   200|  355|
|20151212|   404|    3|
|20151213|   200| 1341|
|20151213|   404|   18|
|20151213|   500|    2|
|20151214|   200|  990|
|20151214|   404|    4|
|20151214|   405|    1|
|20151214|   500|    1|
|20151215|   200|  962|
|20151215|   301|    1|
|20151215|   404|   10|
|20151215|   500|    2|
|20151216|   200| 1123|
|20151216|   304|    1|
|20151216|   404|   46|
|20151216|   500|    1|
|20151217|   200|  923|
|20151217|   301|    1|
|20151217|   304|    1|
+--------+------+-----+
only showing top 20 rows

+--------+------+-----+
|    date|status|count|
+--------+------+-----+
|20151212|   200|  355|
|20151212|   404|    3|
|20151213|   200| 1341|
|20151213|   404|   18|
|20151213|   500|    2|
|20151214|   200|  990|
|20151214|   404|    4|
|20151214|   405|    1|
|20151214|   500|    1|
|20151215|   200|  962|
|20151215|   301|    1|
|20151215|   404|   10|
|20151215|   500|    2|
|20151216|   2

### daily uv

In [24]:
df.groupBy('date').agg(F.countDistinct('host').alias('uv')).sort('date').show()
spark.sql("select date, count(distinct host) as uv from temp_view group by date order by date").show()

+--------+---+
|    date| uv|
+--------+---+
|20151212|120|
|20151213|369|
|20151214|319|
|20151215|278|
|20151216|311|
|20151217|287|
|20151218|210|
|20151219| 54|
|20151220| 70|
|20151221| 69|
|20151222| 65|
|20151223| 85|
|20151224| 87|
|20151225| 56|
|20151226| 39|
|20151227| 38|
|20151228| 40|
|20151229| 40|
|20151230| 36|
|20151231| 38|
+--------+---+
only showing top 20 rows

+--------+---+
|    date| uv|
+--------+---+
|20151212|120|
|20151213|369|
|20151214|319|
|20151215|278|
|20151216|311|
|20151217|287|
|20151218|210|
|20151219| 54|
|20151220| 70|
|20151221| 69|
|20151222| 65|
|20151223| 85|
|20151224| 87|
|20151225| 56|
|20151226| 39|
|20151227| 38|
|20151228| 40|
|20151229| 40|
|20151230| 36|
|20151231| 38|
+--------+---+
only showing top 20 rows



### daily uv (between 20151220 ~ 20151231)

In [25]:
df.filter(df['date'].between('20151220', '20151231')) \
  .groupBy('date').agg(F.countDistinct('host').alias('uv')).sort('date').show()

spark.sql("select date, count(distinct host) as uv from temp_view "
          "where date between '20151220' and '20151231' group by date order by date").show()


+--------+---+
|    date| uv|
+--------+---+
|20151220| 70|
|20151221| 69|
|20151222| 65|
|20151223| 85|
|20151224| 87|
|20151225| 56|
|20151226| 39|
|20151227| 38|
|20151228| 40|
|20151229| 40|
|20151230| 36|
|20151231| 38|
+--------+---+

+--------+---+
|    date| uv|
+--------+---+
|20151220| 70|
|20151221| 69|
|20151222| 65|
|20151223| 85|
|20151224| 87|
|20151225| 56|
|20151226| 39|
|20151227| 38|
|20151228| 40|
|20151229| 40|
|20151230| 36|
|20151231| 38|
+--------+---+



### Explain

In [26]:
lines = spark.sql("explain select count(1) from temp_view where status = '200'").collect()[0]['plan'].split('\n')
for line in lines:
    print line

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[count(1)])
+- Exchange SinglePartition
   +- *(1) HashAggregate(keys=[], functions=[partial_count(1)])
      +- *(1) Project
         +- *(1) Filter (isnotnull(status#8) && (status#8 = 200))
            +- InMemoryTableScan [status#8], [isnotnull(status#8), (status#8 = 200)]
                  +- InMemoryRelation [agent#0, date#1, host#2, identity#3, method#4, protocol#5, referer#6, size#7L, status#8, time#9, url#10, user#11], StorageLevel(disk, memory, deserialized, 1 replicas)
                        +- Scan ExistingRDD[agent#0,date#1,host#2,identity#3,method#4,protocol#5,referer#6,size#7L,status#8,time#9,url#10,user#11]


## UDF

In [27]:
def get_length(s): return len(s)

spark.udf.register('get_length', get_length)
spark.sql("select url, get_length(url) from temp_view").show(truncate=False)

+------------------------+---------------+
|url                     |get_length(url)|
+------------------------+---------------+
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
+----------

In [28]:
udf_get_length = F.udf(get_length)
df.select('url', udf_get_length('url')).show(truncate=False)

+------------------------+---------------+
|url                     |get_length(url)|
+------------------------+---------------+
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
|/administrator/         |15             |
|/administrator/index.php|24             |
+----------

## Hive metastore

In [29]:
spark.sql("""
-- managed table
CREATE TABLE tbl_apache_access_log ( 
  host STRING,
  identity STRING,
  remote_user STRING,
  time TIMESTAMP,
  method STRING,
  url STRING,
  protocol STRING,
  status STRING,
  size BIGINT,
  referer STRING,
  agent STRING)
--PARTITIONED BY (p_time STRING, p_host STRING)
STORED AS TEXTFILE""")

DataFrame[]

In [30]:
!ls -l ./spark-warehouse
# for managed databases and tables

total 0
drwxr-xr-x 2 root root 64 Aug 27 12:39 tbl_apache_access_log
drwxr-xr-x 2 root root 64 Aug 19 16:49 test


In [31]:
spark.sql("LOAD DATA LOCAL INPATH 'access.log' INTO TABLE tbl_apache_access_log")

DataFrame[]

In [32]:
spark.sql("select count(1) from tbl_apache_access_log").show()

+--------+
|count(1)|
+--------+
|   20000|
+--------+



In [33]:
#spark.sql("DROP TABLE tbl_apache_access_log")

## Reference

- https://spark.apache.org/
- https://spark.apache.org/docs/latest/api/sql/index.html
- https://cwiki.apache.org/confluence/display/Hive/LanguageManual
- Spark: The Definitive Guide